## Demo: Part 1 - Benchmarking

In [1]:
using QuadGK              # numerical ∫
using BenchmarkTools

In [2]:
# 1. analytic overlap of two s‑type primitive Gaussians 
# (centers coincide)
# S_G  =  (π / (α + β))^(3/2)
gauss_overlap(α, β) = (π / (α + β))^(3/2)

gauss_overlap (generic function with 1 method)

In [3]:
# 2. numerical overlap of two Slater 1s functions (centres coincide)
#    Normalised STO: χ(r) = (ζ^3/π)^{1/2} * exp(-ζ r)
function slater_overlap(ζ1, ζ2)
    pref = (ζ1^3 * ζ2^3)^(1/2) / π
    integrand(r) = 4π * pref * exp(-(ζ1 + ζ2) * r) * r^2
    QuadGK.quadgk(integrand, 0.0, Inf; atol = 1e-12, rtol = 1e-12)[1]
end

slater_overlap (generic function with 1 method)

In [4]:
# --- demo parameters ---------------------------------------------------------
α  = 1.24     # Gaussian exponent
β  = 0.75
ζ1 = 1.24     # Slater effective charges
ζ2 = 0.75

0.75

In [5]:
println("Single‑shot results:")
println("  Gaussian overlap  = ", gauss_overlap(α, β))
println("  Slater  overlap   = ", slater_overlap(ζ1, ζ2))

Single‑shot results:
  Gaussian overlap  = 1.9835593267717158
  Slater  overlap   = 0.910448072053756


In [6]:
println("\nTiming (100 000 evaluations each):")
@btime gauss_overlap($α, $β)        setup=() evals=100000
@btime slater_overlap($ζ1, $ζ2)     setup=() evals=100000


Timing (100 000 evaluations each):
  7.593 ns (0 allocations: 0 bytes)
  982.040 ns (3 allocations: 384 bytes)


0.910448072053756

## Demo: Part 2 - Test

In [7]:
using BasisSets
using OohataHuzinaga

In [10]:
mol = molecule("/Users/leticiamadureira/Projects/BasisSets.jl/test/data/hydrogen/h-atom.xyz")

Molecule(["H", "H"], [0.0 0.0 0.0; 0.0 0.0 0.74], [1, 1])

In [ ]:
shell_table = Dict(
           "H" => [(1, 0, 3)])


Dict{String, Vector{Tuple{Int64, Int64, Int64}}} with 1 entry:
  "H" => [(1, 0, 3)]

In [12]:
basis = BasisSets.optimizebasis(mol, shell_table)

Optimizing basis for atom: H
Optimizing for n=1, l=0, k=3
[0.24999999999999997 0.7999999999999999 2.56] ℓ = 0
 m = 0
 n = 0
Optimizing basis for atom: H
Optimizing for n=1, l=0, k=3
[0.24999999999999997 0.7999999999999999 2.56] ℓ = 0
 m = 0
 n = 0


2-element Vector{BasisSets.GaussianBasisSet}:
 BasisSets.GaussianBasisSet([0.0 0.0 0.0], [0.24999999999999997 0.7999999999999999 2.56], [0.8000760295486874 0.040527319782266 0.8953541965717289], [0.2519794355383807 0.602875426920206 1.442414455797365], 3, 0, 0, 0)
 BasisSets.GaussianBasisSet([0.0 0.0 0.74], [0.24999999999999997 0.7999999999999999 2.56], [0.8000760295486874 0.040527319782266 0.8953541965717289], [0.2519794355383807 0.602875426920206 1.442414455797365], 3, 0, 0, 0)

In [17]:
basis[1]

BasisSets.GaussianBasisSet([0.0 0.0 0.0], [0.24999999999999997 0.7999999999999999 2.56], [0.8000760295486874 0.040527319782266 0.8953541965717289], [0.2519794355383807 0.602875426920206 1.442414455797365], 3, 0, 0, 0)

In [13]:
res=OohataHuzinaga.rhf(basis,mol)


Overlap is done!
Kinetic is done!
Attraction is done!
Repulsion is done!
HCore is done!
Starting SCF iterations...
-1.5791451546041873
1.3513513513513513
-0.5854539917056862
1.3513513513513513


Results(-0.5854539917056862, -1.9368053430570376, 1.3513513513513513, [0.26357151422477454 0.26357151422477454; 0.26357151422477454 0.26357151422477454])

In [14]:
res.energy

-1.9368053430570376